In [ ]:
import MEArec as mr
import MEAutility as mu
from pprint import pprint

In [ ]:
# define cell_models folder
cell_folder = '../cell_models/bbp'

# generate templates with default params (n=20) 
# the templates are not saved, but the intracellular simulations are saved in 'templates_folder'
tempgen = mr.gen_templates(cell_folder, params={'n':20}, templates_folder='../data/templates')

# this will take a few minutes...

In [ ]:
# save templates in h5 format
mr.save_template_generator(tempgen, filename='test_templates.h5')

# save in folder format
mr.save_template_generator(tempgen, filename='test_templates')

# load back template generator
tempgen_loaded = mr.load_templates('test_templates.h5')

In [ ]:
# pass h5, folder, or template_generator object

recgen = mr.gen_recordings(templates='test_templates.h5')
# recgen_1 = mr.gen_recordings(templates='test_templates')
# recgen_2 = mr.gen_recordings(tempgen=tempgen)


In [ ]:
# save recordings in h5 format
mr.save_recording_generator(recgen, filename='test_recordings.h5')

# save recordings in h5 format
mr.save_recording_generator(recgen, filename='test_recordings')

# load back recording generator
recgen_load = mr.load_recordings('test_recordings')

In [ ]:
# plot template
probe_name = recgen.info['recordings']['electrode_name']
mea = mu.return_mea(probe_name)

mu.plot_mea_recording(tempgen.templates[0], mea)

In [ ]:
mu.plot_mea_recording(recgen.recordings, mea)